In [1]:
#https://keon.io/deep-q-learning/
#에서 참고


C:\Users\sh2\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from collections import deque
import numpy as np
import gym
import random

In [81]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen =2000)
        self.gamma = 0.95 # 보상이 느릴수록 페널티를 주기위해
        self.epsilon = 1.0 #e
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.target_model = self.build_model()
        self.main_model = self.build_model()
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(128,input_dim=self.state_size,activation='relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'softmax'))
        model.compile(loss='mse' , optimizer = Adam(lr=self.learning_rate))
        return model
    def remember(self, state, action, rewar, next_state, done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            
            return random.randrange(self.action_size)
        
        act_values = self.main_model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        #memory 의 길이보다 batch_size 가 작게 들어온 경우 return
        x_stack = np.empty(0).reshape(0,self.state_size)
        y_stack = np.empty(0).reshape(0,self.action_size)
        if(len(self.memory) <= batch_size):
            return
        minibatch = random.sample(self.memory, batch_size)

        #target_model
        #main_model
        for state, action, reward, next_state, done in minibatch:
            Q = self.main_model.predict(state)
            
            if done:
                Q[0,action] = reward
                
            else :
                Q[0,action] = reward + self.gamma * np.max(self.target_model.predict(next_state))
            
           
            
            y_stack = np.vstack([y_stack,Q])
            x_stack = np.vstack([x_stack,state])
            
        

        self.main_model.fit(x_stack,y_stack)
        
        if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
                


In [82]:
env = gym.make('CartPole-v0')

    
agent = DQNAgent(4,2)

In [83]:

episodes = 1000
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
        
    state = np.reshape(state, [1, 4])
      
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible until score of 500
        # the more time_t the more score
        
    for time_t in range(10001):
            # turn this on if you want to render
            #env.render()
            # Decide action
        action = agent.act(state)
            # Advance the game to the next frame based on the action.
            # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 4])
            # Remember the previous state, action, reward, and done
        if done:
            reward = -100
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
                
            print("episode: {}/{}, score: {}"
                  .format(e, episodes, time_t))
            break
    
    if e % 10 == 1:
        for _ in range(50):
            agent.replay(24)
    agent.target_model.set_weights(agent.main_model.get_weights())    

episode: 0/1000, score: 15
episode: 1/1000, score: 8
Epoch 1/1
24/24 [==============================] - 1s 28ms/step - loss: 421.2569
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.2283
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.2005
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.1741
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.1452
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.1326
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.1136
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.0895
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.1034
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.5310
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.0952
Epoch 1/1
24/24 [==============================] - 0s 83

Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 629.6043
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 208.8039
episode: 12/1000, score: 10
episode: 13/1000, score: 7
episode: 14/1000, score: 9
episode: 15/1000, score: 9
episode: 16/1000, score: 29
episode: 17/1000, score: 16
episode: 18/1000, score: 13
episode: 19/1000, score: 13
episode: 20/1000, score: 9
episode: 21/1000, score: 15
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7777
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5752
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.3508
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7250
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 417.5933
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 421.5653
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.8586
Ep

Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 421.4419
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.4256
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 417.4255
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.1894
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.1463
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 629.9138
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.9114
episode: 32/1000, score: 15
episode: 33/1000, score: 9
episode: 34/1000, score: 16
episode: 35/1000, score: 8
episode: 36/1000, score: 12
episode: 37/1000, score: 8
episode: 38/1000, score: 10
episode: 39/1000, score: 11
episode: 40/1000, score: 8
episode: 41/1000, score: 8
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 425.7967
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 838

Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 846.5356
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7856
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.7410
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 634.1150
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7904
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.2297
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 0.7439
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 630.2314
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.1199
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 633.9645
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.5705
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.2465
episode: 52/1000, score: 10
episo

Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 209.8536
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 629.8447
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 630.5405
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 627.4232
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 418.4738
Epoch 1/1
24/24 [==============================] - 0s 124us/step - loss: 210.0981
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.8898
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 209.5028
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.9708
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 0.9067
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 630.1614
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 209.6180
Epoch 1/1
24/24 [============

...

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.6169
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 630.0550
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.6128
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.8032
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 625.8896
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 422.4570
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 209.1884
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 632.7073
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.3611
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.8297
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 1050.8384
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.8340
Epoch 1/1
24/24 [============

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.9512
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.5770
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.2984
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 1263.2501
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.0635
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 417.4954
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.6221
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.5561
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 626.1741
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 633.8324
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 637.6550
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 209.1953
Epoch 1/1
24/24 [==========

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7042
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 629.8964
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 421.3962
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 421.8813
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 209.0515
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 213.3509
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.9171
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 419.5570
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.2001
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 628.0370
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 834.1321
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 842.4651
Epoch 1/1
24/24 [========

episode: 968/1000, score: 11
episode: 969/1000, score: 9
episode: 970/1000, score: 11
episode: 971/1000, score: 8
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 1058.6698
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 638.5085
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 842.4642
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 1048.2301
Epoch 1/1
24/24 [==============================] - 0s 147us/step - loss: 417.5132
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.2606
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 213.0823
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 422.1772
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 843.8392
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.7401
Epoch 1/1
24/24 [==============================] - 0s 125us/step - los

Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 637.9773
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.5105
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 209.2827
Epoch 1/1
24/24 [==============================] - 0s 124us/step - loss: 842.1818
episode: 982/1000, score: 11
episode: 983/1000, score: 9
episode: 984/1000, score: 10
episode: 985/1000, score: 8
episode: 986/1000, score: 11
episode: 987/1000, score: 8
episode: 988/1000, score: 10
episode: 989/1000, score: 9
episode: 990/1000, score: 12
episode: 991/1000, score: 10
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 637.9926
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 425.6721
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 417.2999
Epoch 1/1
24/24 [==============================] - 0s 208us/step - loss: 213.3563
Epoch 1/1
24/24 [==============================] - 0s 125us/ste